In [1]:
import tensorflow as tf
import numpy as np


In [ ]:
# How to perform operation on arrays.
elems = np.array([1, 2, 3, 4, 5, 6])
squares = tf.map_fn(lambda x: x * x, elems)
# squares == [1, 4, 9, 16, 25, 36]

sess=tf.Session()
print(sess.run(squares))

In [ ]:
elems = (np.array([1, 2, 3]), np.array([-1, 1, -1]))
alternate = tf.map_fn(lambda x: x[0] * x[1], elems, dtype=tf.int64)
# alternate == [-1, 2, -3]

sess=tf.Session()
print(sess.run(alternate))

In [ ]:
# "Broadcasting ability of TF"
import tensorflow as tf

# Reset graph - just in case.
tf.reset_default_graph()

x = tf.constant([[0, 1,2],[2, 3,2],[4, 5,2],[6, 7,2]], dtype=tf.float32)
y = tf.constant([[0, 1,2],[-2, -1,2]], dtype=tf.float32)

x_vect_len = int(x.shape[1])
print("len=",x_vect_len)
x_ = tf.expand_dims(x, 0)
y_ = tf.expand_dims(y, 1)
z = tf.reshape(tf.add(x_, y_), [-1, x_vect_len])

sess = tf.InteractiveSession()
sess.run(z)
print(x_.eval())
print(y_.eval())

sess.close()

In [ ]:
# Reset graph - just in case.
tf.reset_default_graph()

# place holders 
batch = tf.placeholder(tf.float32, shape=[3,3], name="batch")
memory = tf.placeholder(tf.float32, shape=[2,3], name="memory")
# Cos similarity
norm_batch = tf.nn.l2_normalize(batch,0) 
norm_memory = tf.nn.l2_normalize(memory,0)

nb_ = tf.expand_dims(norm_batch, 0)
nm_ = tf.expand_dims(norm_memory, 1)
cos_similarity = tf.reshape(tf.add(nb_,nm_), [-1, 2])
#cos_similarity = tf.reshape(tf.reduce_sum(tf.multiply(nb_,nm_)), [-1, 2])

                            
sess=tf.InteractiveSession()
cos_sim=sess.run(cos_similarity,feed_dict={batch:[[1,2,3],[3,3,3],[1,2,6]],memory:[[2,4,6],[1,2,4]]})

print(cos_sim)

sess.close()

In [ ]:
# Reset graph - just in case.
tf.reset_default_graph()

# Size of the hidden state 64
HIDDEN_SIZE = 3

# Size of the MANN memory.
MEMORY_SIZE = 5

# A batch size of 100
BATCH_SIZE = 2

# A single recurrent layer of number of units = sequences of length
# e.g. 200 bytes
SEQ_LENGTH = 10

EPS = 1e-15

# place holders 
batch = tf.placeholder(tf.float32, shape=None, name="batch")
#memory = tf.placeholder(tf.float32, shape=[None], name="memory")
memory = tf.Variable(tf.zeros([MEMORY_SIZE, HIDDEN_SIZE]), trainable=False, name="memory")

memory_set = memory.assign([[1, 0, 1],[0.1, 0.2 , 0.4],[ 0,0,0],[-0.3,0.2,0.3],[0, 1, 0]])

# Create BATCH_SIZE placeholders for similarity - each MEMORY_SIZE x 1,  
# 0. Placeholders for inputs.
with tf.name_scope("similarity"):
  # Define similarity buffers.
  similarity = list()
  #for b in range(BATCH_SIZE):
    # Collect placeholders for similarity.
    #similarity.append(tf.placeholder(tf.float32, shape=[MEMORY_SIZE, 1], name="Similarity"))
 
  # Normalize
  norm_batch = tf.nn.l2_normalize(batch,1) 
  norm_memory = tf.nn.l2_normalize(memory,1)
  print(norm_batch[0])
  # Define similarity buffers.
  numerator_batch = list()
  denominator_batch = list()
  similarity_batch = list()

  for b in range(BATCH_SIZE):
    similarity_batch.append(tf.map_fn(lambda x: tf.reduce_sum(tf.multiply(norm_batch[b],x)), norm_memory))
    # Read weight based on similarity.
  read_weight = tf.nn.softmax(similarity_batch)
    
sess=tf.InteractiveSession()
# Initialize.
sess.run(memory_set)
print("memory =",memory.eval())

tmp_batch = [[1, 0, 1],[-0.3,0.3,0.3]]

print("Batch=",batch.eval(feed_dict={batch:tmp_batch}))
print("Norm batch=",norm_batch.eval(feed_dict={batch:tmp_batch}))


num, den, sim, rw=sess.run([numerator_batch, denominator_batch, similarity_batch, read_weight],
                           feed_dict={batch:tmp_batch})
print("norm memory =",norm_memory.eval())

print("numerator_batch=",num)
print("denominator_batch=\n",den,"\n")

print("similarity=",sim)
print("rw=", rw)


sess.close()


In [42]:
# Different approach - change memory "orientation"

# Reset graph - just in case.
tf.reset_default_graph()

# Size of the hidden state 64
HIDDEN_SIZE = 3

# Size of the MANN memory.
MEMORY_SIZE = 5

# A batch size of 100
#BATCH_SIZE = 2

# A single recurrent layer of number of units = sequences of length
# e.g. 200 bytes
SEQ_LENGTH = 10

# "Read decay".
GAMMA = 0.9

#EPS = 1e-15

# place holders 
batch = tf.placeholder(tf.float32, shape=None, name="Batch_h")
#memory = tf.placeholder(tf.float32, shape=[None], name="memory")
memory = tf.Variable(tf.zeros([HIDDEN_SIZE, MEMORY_SIZE]), trainable=False, name="Memory_M")
#alpha = tf.Variable(tf.truncated_normal(shape=[1]), name="Alpha")
alpha = tf.Variable(tf.truncated_normal(shape=[1]), name="Alpha")

# SET INITIAL MEMORY STATE.
memory_set = memory.assign(tf.transpose([[1, 0, 1],
                            [0.1, 0.2, 0.4],
                            [ 0, 0.3, 0],
                            [-0.3, 0.2, 0.3],
                            [0, 1, 0]]))
alpha_set = alpha.assign([100])

# Placeholders for previous weights.
prev_update_weights = tf.placeholder(tf.float32, shape=None, name="Prev_uw")
prev_read_weights = tf.placeholder(tf.float32, shape=None, name="Prev_rw")


with tf.name_scope("Read_head"):
    # Normalize batches and memory.
    norm_batch = tf.nn.l2_normalize(batch,1, name="NormalizedBatch_h") 
    norm_memory = tf.nn.l2_normalize(memory,1, name="NormalizedMemory_h")

    # calculate similarity.
    similarity = tf.tensordot(norm_batch, norm_memory, axes=1, name= "Similarity_D") 
    # Read weights based on similarity.
    read_weights = tf.nn.softmax(similarity, name="Read_weights_rw")
    # Read "vector" (in fact batch).
    r = tf.tensordot(read_weights, tf.transpose(memory), axes=1, name="Read_vector_r")

# TODO: add dependencies, that write will be done after read.
with tf.name_scope("Write_head"):
    write_weights = tf.add(tf.sigmoid(alpha) * prev_read_weights, 1,# (1 - tf.sigmoid(alpha)) * read_weights, 
                           name="Write_weights_ww")
    
    calculate_mem_update = tf.tensordot(tf.transpose(batch), write_weights, axes=1)
    memory_update_op = memory.assign(memory + calculate_mem_update)

with tf.name_scope("Update_head"): # This relies on prev. weights and will be used in fact in NEXT step.
    update_weights = tf.add(GAMMA * prev_update_weights, read_weights, name="Update_weights_uw")

    
    
# Finally - initialize all variables.
initialize_model = tf.global_variables_initializer()    
    
# Execute graph.
sess=tf.InteractiveSession()
# Initialize.
sess.run(initialize_model)
sess.run([memory_set, alpha_set])
print("Memory =\n",memory.eval())

tmp_batch = [[1, 0, 1],[-0.3, 0.3, -0.3]]
# Prev UW [batch size x memory size]
prev_uw = [[0,0,0,0,0], [0,0,0,0,0]]
prev_rw = [[0,0,0,0,0], [0,0,0,0,0]]

print("\nBatch=\n",batch.eval(feed_dict={batch:tmp_batch}))
#print("Norm batch=",norm_batch.eval(feed_dict={batch:tmp_batch}))

for i in range(10):
    sim, r_vect, prev_rw, prev_uw, prev_ww, mu_op = sess.run(
        [similarity, r, read_weights, update_weights, write_weights, memory_update_op],
                               feed_dict={
                                   batch:tmp_batch,
                                   prev_update_weights: prev_uw,
                                   prev_read_weights: prev_rw
                               })
    #print("Norm memory =\n",norm_memory.eval())
    print ("\n=====i=", i)
    print("Memory after update=\n", mu_op)
    #print("\nSimilarity=\n",sim)
    print("\nprev_rw=\n", prev_rw)
    #print("\nr_vect=\n", r_vect)
    #print("\nprev_uw=\n", prev_uw)
    print("\nprev_ww=\n", prev_ww)



sess.close()

Memory =
 [[ 1.          0.1         0.         -0.30000001  0.        ]
 [ 0.          0.2         0.30000001  0.2         1.        ]
 [ 1.          0.40000001  0.          0.30000001  0.        ]]

Batch=
 [[ 1.          0.          1.        ]
 [-0.30000001  0.30000001 -0.30000001]]

=====i= 0
Memory after update=
 [[ 1.70000005  0.80000001  0.69999999  0.39999998  0.69999999]
 [ 0.30000001  0.5         0.60000002  0.5         1.29999995]
 [ 1.70000005  1.10000002  0.69999999  1.          0.69999999]]

prev_rw=
 [[ 0.45834252  0.17094871  0.12408432  0.12254006  0.12408432]
 [ 0.06612217  0.16460168  0.22554328  0.21601786  0.32771501]]

prev_ww=
 [[ 1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.]]

=====i= 1
Memory after update=
 [[ 2.83850598  1.6215682   1.45642138  1.15773475  1.42576981]
 [ 0.61983669  0.84938049  0.96766305  0.86480534  1.69831443]
 [ 2.83850598  1.92156827  1.45642138  1.75773478  1.42576981]]

prev_rw=
 [[ 0.3082411   0.19331703  0.16681786  0.16480623  0.166817